# Turbo Show
Eyes + gentle head wiggle + subtle body fidget + speech + moves.

In [1]:
import time, threading, importlib
import robot_moves as rm, eyes_lib as elib, camera_lib as camlib, tts_lib as tts
importlib.reload(rm); importlib.reload(elib); importlib.reload(camlib); importlib.reload(tts)
eyes = elib.get_eyes() if hasattr(elib,"get_eyes") else getattr(elib,"eyes",None) or elib.Eyes()
cam  = camlib.get_cam() if hasattr(camlib,"get_cam") else getattr(camlib,"cam",None) or camlib.Camera()
rm.set_base_speed(200); rm.set_rate(30)
print("Libraries ready.")

camera_lib ready: cam.nod(), cam.shake(), cam.wiggle(), cam.tiny_wiggle(), cam.center_all(), cam.glance_left/right(), cam.look_up/down()
eyes_lib ready → topic=/sonar_controller/set_rgb indices=(0, 1)
Libraries ready.


In [2]:
EYE_COLOR=(253,208,0)
BLINK_ON_S=0.10; BLINK_OFF_S=0.10
def blink_big(duration_s=1.0, open_color=EYE_COLOR):
    end=time.time()+float(duration_s); on=True
    while time.time()<end:
        (eyes.set_both(*open_color) if on else eyes.off())
        on=not on; time.sleep(BLINK_ON_S if on else BLINK_OFF_S)
    eyes.set_both(*open_color)

def head_wiggle_gentle(seconds=2.0):
    cam.wiggle(cycles=1, amplitude=120)

def body_fidget_small(seconds=2.0):
    t_end=time.time()+float(seconds)
    while time.time()<t_end:
        rm.left(0.10, speed=150); rm.stop(); time.sleep(0.05)
        rm.right(0.10, speed=150); rm.stop(); time.sleep(0.05)
        rm.forward(0.08, speed=140); rm.stop(); time.sleep(0.05)
        rm.backward(0.08, speed=140); rm.stop(); time.sleep(0.05)

In [3]:
PROMO=("Key ora! I'm Turbo, I'm a robot from Mata moe e. "
       "We're building something big in twenty twenty-six. "
       "Robots, and A I. Stay tuned for more!")
try:
    wav_path = tts.pre_synth(PROMO, voice="ryan", length_scale="0.98", sentence_silence="0.08")
    print("Cached at", wav_path)
except Exception as e:
    print("TTS cache warning:", e); wav_path=None

def speak_line_now():
    if wav_path and hasattr(tts,"play_path_async"):
        tts.play_path_async(wav_path, device=None)
    elif wav_path:
        tts.play_wav_async(wav_path, device=None)
    else:
        path = tts.synth_to_wav(PROMO, voice="ryan", length_scale="0.98", sentence_silence="0.08")
        tts.play_wav_async(path, device=None)

def speak_with_blink():
    threading.Thread(target=head_wiggle_gentle, args=(2.2,), daemon=True).start()
    threading.Thread(target=body_fidget_small, args=(2.2,), daemon=True).start()
    blink_big(0.8, open_color=EYE_COLOR)
    speak_line_now()
    threading.Thread(target=lambda:(time.sleep(2.0), blink_big(0.8, open_color=EYE_COLOR)), daemon=True).start()

Cached at /tmp/piper-65601b428923cc8a.wav


In [4]:
# --- Fix eyes publisher for sonar "eyes" ---
import time, rclpy
from rclpy.node import Node
from ros_robot_controller_msgs.msg import RGBStates, RGBState

EYES_TOPIC = "sonar_controller/set_rgb"   # change if your topic is different
EYES_INDEXES = (0, 1)                     # left=0, right=1 on your robot

class _Eyes(Node):
    def __init__(self, topic=EYES_TOPIC):
        if not rclpy.ok():
            rclpy.init(args=None)
        super().__init__("eyes_fix_pub")
        self.pub = self.create_publisher(RGBStates, topic, 10)

    def set_both(self, r,g,b):
        msg = RGBStates()
        msg.states = [RGBState(index=EYES_INDEXES[0], red=int(r), green=int(g), blue=int(b)),
                      RGBState(index=EYES_INDEXES[1], red=int(r), green=int(g), blue=int(b))]
        self.pub.publish(msg)
        time.sleep(0.03)  # small flush for notebook

    def off(self): self.set_both(0,0,0)

    def blink_color(self, open_rgb=(253,208,0), closed_rgb=(0,0,0), total_s=0.9, hold_open=0.35, hold_closed=0.20):
        # Big, visible “blink” using color on/off timing
        t0 = time.time()
        # close
        self.set_both(*closed_rgb); time.sleep(hold_closed)
        # open
        self.set_both(*open_rgb);   time.sleep(hold_open)
        # if time left, stay open
        rem = total_s - (hold_open + hold_closed)
        if rem > 0: time.sleep(rem)

try:
    eyes_node  # reuse if already created
except NameError:
    eyes_node = _Eyes()
    print("Eyes ready on", EYES_TOPIC)


Eyes ready on sonar_controller/set_rgb


In [11]:
def perform_show():
    eyes.set_both(*EYE_COLOR); cam.center_all(); time.sleep(0.2)
    eyes.wink("left", color=(255,255,255), on_s=0.12, off_s=0.10, repeats=1)
    eyes.wink("right", color=(255,255,255), on_s=0.12, off_s=0.10, repeats=1)
    rm.turn_left(2.4); rm.stop(); time.sleep(0.5)
    if hasattr(rm,"drift_left_wheels"):
        rm.drift_left_wheels(0.6); rm.drift_right_wheels(0.6); rm.stop(); time.sleep(0.1)
    rm.forward(0.5); rm.stop(); time.sleep(0.2)
    speak_with_blink()
    time.sleep(9.0); cam.nod()
    try: rm.horn()
    except Exception as e: print("[horn] warn:", e)
    time.sleep(1.0);
    rm.forward(0.6); rm.stop(); eyes.off()
    print("Show complete.")
print("Ready. Run perform_show() next.")

Ready. Run perform_show() next.


In [12]:
perform_show()

Show complete.


In [7]:
# Eyes should go amber, then off, then blink once
eyes_node.set_both(253,208,0)   # amber like Pixar
time.sleep(0.5)
eyes_node.off()
time.sleep(0.2)
eyes_node.blink_color(open_rgb=(253,208,0), total_s=1.0)